In [1]:
import requests
from bs4 import BeautifulSoup
import httpx
import json
import matplotlib.pyplot as plt
from langchain.text_splitter import RecursiveCharacterTextSplitter
from supabase import create_client
import PyPDF2
import nltk
from nltk.tokenize import word_tokenize
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.corpus import stopwords
import os
import openai
import json
from IPython.display import display
from sklearn.metrics.pairwise import cosine_similarity


Could not import azure.core python package.


In [2]:
# URL of the page you want to scrape
url = 'https://www.geeksforgeeks.org/machine-learning/'


# Send GET request to the webpage
response = requests.get(url)

In [ ]:

# Initialize the Supabase client
supabase_url = "https://mufqacshyjgmzivznuwo.supabase.co"
supabase_key = "Please add your key here"
client = create_client(supabase_url, supabase_key)

# change the table name as required
table_name = "combined_test" 

headers = {
    "apikey": supabase_key,
    "Content-Type": "application/json"
}

endpoint = f"{supabase_url}/rest/v1/combined_test"

# Please add your open ai key
openai.api_key = "Please add your key here"

In [4]:
def preprocess_sentence(sentence):
    stop_words = set(stopwords.words('english'))
    # Tokenize the sentence
    tokens = word_tokenize(sentence.lower())
    # Remove non-alphabetic characters and stopwords
    tokens = [word for word in tokens if word.isalpha() and word not in stop_words]
    return tokens

# Download required NLTK resources, if not already downloaded
def download_nltk_resources():
    nltk.download('punkt')      # For tokenization
    nltk.download('stopwords')  # For removing stop words

# Ensure NLTK resources are downloaded before proceeding
download_nltk_resources()


def extract_text_by_page(pdf_path):
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        for i, page in enumerate(reader.pages):
            # Extract text from page and preprocess
            text = page.extract_text()
            sentences = nltk.sent_tokenize(text)
            # Preprocess each sentence and combine into one list of words for the page
            words = [preprocess_sentence(sentence) for sentence in sentences]
            words = [word for sublist in words for word in sublist]  # Flatten the list
            yield TaggedDocument(words=words, tags=[str(i)])
            
pdf_path = 'C:/Users/devas/Downloads/AI-book.pdf'
tagged_data = list(extract_text_by_page(pdf_path))

### Divide the data in chunks of size 1000
def chunk_section(section, chunk_size, chunk_overlap):
    text_splitter = RecursiveCharacterTextSplitter(
        separators=["\n\n", "\n", " ", ""],
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len)
    chunks = text_splitter.create_documents(
        texts=[section["text"]], 
        metadatas=[{"source": section["source"]}])
    return [{"text": chunk.page_content, "source": chunk.metadata["source"]} for chunk in chunks]

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\devas\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\devas\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
model = Doc2Vec(vector_size=50, window=5, min_count=5, workers=4, epochs=100)
model.build_vocab(tagged_data)
model.train(tagged_data, total_examples=model.corpus_count, epochs=model.epochs)

# Save the model
model.save("jesc102_model.d2v")

2023-12-01 14:10:11,422:INFO - Doc2Vec lifecycle event {'params': 'Doc2Vec(dm/m,d50,n5,w5,mc5,s0.001,t4)', 'datetime': '2023-12-01T14:10:11.422960', 'gensim': '4.1.2', 'python': '3.9.13 (main, Aug 25 2022, 23:51:50) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.22621-SP0', 'event': 'created'}
2023-12-01 14:10:11,425:INFO - collecting all words and their counts
2023-12-01 14:10:11,427:INFO - PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2023-12-01 14:10:11,571:INFO - collected 28892 word types and 1153 unique tags from a corpus of 1153 examples and 281697 words
2023-12-01 14:10:11,572:INFO - Creating a fresh vocabulary
2023-12-01 14:10:11,647:INFO - Doc2Vec lifecycle event {'msg': 'effective_min_count=5 retains 6246 unique words (21.618441090959436%% of original 28892, drops 22646)', 'datetime': '2023-12-01T14:10:11.647520', 'gensim': '4.1.2', 'python': '3.9.13 (main, Aug 25 2022, 23:51:50) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.

2023-12-01 14:10:16,949:INFO - EPOCH - 12 : training on 281697 raw words (239982 effective words) took 0.4s, 611899 effective words/s
2023-12-01 14:10:17,331:INFO - worker thread finished; awaiting finish of 3 more threads
2023-12-01 14:10:17,337:INFO - worker thread finished; awaiting finish of 2 more threads
2023-12-01 14:10:17,341:INFO - worker thread finished; awaiting finish of 1 more threads
2023-12-01 14:10:17,350:INFO - worker thread finished; awaiting finish of 0 more threads
2023-12-01 14:10:17,351:INFO - EPOCH - 13 : training on 281697 raw words (239910 effective words) took 0.4s, 603573 effective words/s
2023-12-01 14:10:17,735:INFO - worker thread finished; awaiting finish of 3 more threads
2023-12-01 14:10:17,739:INFO - worker thread finished; awaiting finish of 2 more threads
2023-12-01 14:10:17,743:INFO - worker thread finished; awaiting finish of 1 more threads
2023-12-01 14:10:17,751:INFO - worker thread finished; awaiting finish of 0 more threads
2023-12-01 14:10:17,

KeyboardInterrupt: 

In [ ]:
model = Doc2Vec.load("jesc102_model.d2v")

In [ ]:
sections= []
if response.status_code == 200:
    # Parse the HTML content of the page
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Find the <ul> tag with the class 'leftBarList'
    left_bar_list = soup.find('ul', class_='leftBarList')
    
    # Find all <a> tags within the left bar list
    links = left_bar_list.find_all('a') if left_bar_list else []

    # Loop through all found <a> tags to get their href attribute (URLs)
    for link in links:
        # Ensure that only full URLs are printed
        href = link.get('href')
        full_url = href if href.startswith('http') else f"https://www.geeksforgeeks.org{href}"
        
        page_response = requests.get(full_url)
        if page_response.status_code == 200:
            # Parse the HTML content of the page
            page_soup = BeautifulSoup(page_response.content, 'html.parser')
            
             # Find all <p> tags on the page and get their text content
            paragraphs = page_soup.find_all('p')
            paragraph_texts = [p.get_text(separator='\n', strip=True) for p in paragraphs]
            
            # Combine all paragraph texts into a single string
            full_text = '\n\n'.join(paragraph_texts)
            section = {"text": full_text, "source": full_url}
            sections.append({"text": full_text, "source": full_url})
            chunks_ds = chunk_section(section,chunk_size=chunk_size, chunk_overlap=chunk_overlap)
            for chunk in chunks_ds:
                vector = model.infer_vector(preprocess_sentence(chunk["text"])).tolist()
                vector_data = json.dumps(vector)
            
                data_to_insert = {
                    "source": chunk["source"],
                    "text" : chunk["text"],
                    "vector": vector_data 
                }
            
            response = httpx.post(endpoint, headers=headers, json=data_to_insert)
            
        else:
            print(f"Failed to retrieve the page from {full_url}. Status code: {page_response.status_code}")
else:
    print(f"Failed to retrieve the webpage. Status code: {response.status_code}")

In [ ]:
def time_decorator(func):
    def wrapper(*args, **kwargs):
        start_time = time.time()
        result = func(*args, **kwargs)
        end_time = time.time()
        elapsed_time = end_time - start_time
        print(f"{func.__name__} took {elapsed_time:.6f} seconds to execute.")
        return result
    return wrapper

@time_decorator
def continue_conversation(prompt):
    response = openai.ChatCompletion.create( model="gpt-4", top_p=0,
                                        messages=prompt,
                                        temperature=0 )
    return response['choices'][0]['message']['content'].strip()


In [ ]:
# Supabase details
url: str = "https://mufqacshyjgmzivznuwo.supabase.co"
key: str = "Please add your key here"

# Create a client to connect to Supabase
supabase: Client = create_client(url, key)

import ast
import numpy as np
import pandas as pd

ids = []
sources = []
vectors = []

response = supabase.table("vector_final").select("id, source, vector").execute()
if response.data:
    for row in response.data:
        # Extracting each column's value and appending to the respective lists
        ids.append(row['id'])
        sources.append(row['source'])
        vectors.append(row['vector'])  # Assuming 'vector' is already in the desired format (e.g., list of numbers)

# Now, ids, sources, and vectors have the data from their respective columns.
vectors = [ast.literal_eval(vector) for vector in vectors]
vectors = np.array(vectors)

In [ ]:
truth = pd.read_excel('llm_testing.xlsx')
test = pd.read_excel('questions.xlsx')

In [ ]:
answers = []
for q in test["Question"]:
    top5_data = []
    initial_user_prompt= q
    new_system_prompt ="Answer the question asked by the user" 

    print(q)
    
    history = [
        {"role": "system", "content": new_system_prompt},
        {"role": "user", "content": initial_user_prompt}]

    response_by_LLM = continue_conversation(history)
    vector_sentence = model.infer_vector(preprocess_sentence(response_by_LLM))
    sr = []
    
    vector_sentence = np.array(vector_sentence)
    vector_sentence_2d = vector_sentence.reshape(1, -1)

    similarities = cosine_similarity(vector_sentence_2d, vectors)

    # Find the index of the most similar paragraph
    most_similar_index = np.argmax(similarities)

    top_5_indexes = np.argsort(similarities[0])[-5:][::-1]
    for index in top_5_indexes:
        sr.append(sources[index])

    res = []
    for i in range(len(sr)):
        data = supabase.table("vector_final").select("text").eq("source", sr[i]).execute()
        top5_data.append(data)

        # Assuming vectors are stored as arrays and not serialized
        relevant_text_from_source = [row['text'] for row in top5_data[i].data]
    
        initial_user_prompt= q
        
        new_system_prompt = f"Compare the following response with the provided source text using semantic check and determine its accuracy. " \
                     f"Response: '{response_by_LLM}'. " \
                     f"Source Text: '{relevant_text_from_source}'. " \
                     f"If the response accurately reflects the source text, reply 'accurate'. " \
                     f"If the response does not accurately reflect the source text , reply 'inaccurate'. " \
                     f"If the response's content is not found in the source text, reply 'information not found in source'."\
                     f"If the response is not relevant to question, reply 'response is irrelevant to question'. "


    
        history = [
            {"role": "system", "content": new_system_prompt},
            {"role": "user", "content": initial_user_prompt}]

        response = continue_conversation(history)
        res.append(response)
        print(res)
    if 'Accurate' in res:
        answers.append("Final check: Accurate")
    else:
        answers.append("Final check: "+ response)


In [ ]:
acc = 1
for i in range(len(answers)):
    print(truth['Answer'][i])
    print(answers[i])
    if (truth['Answer'][i] == answers[i]):
        acc += 1
print("Accuracy : ", acc/ len(answers))